In [ ]:
import cv2
import torch
from ultralytics import YOLO

from common.yolo.yolo_results_aux import results_2_pose_list
from common.yolo.nms import nms_for_yolo_pose

from common.tracker.sort_tracker import SortTracker

In [ ]:
# Load the YOLOv8n-Pose model
model = YOLO("/opt/models/yolo/yolov8n-pose.pt")

# Test whether the GPU device is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Move the model to the device
model.to(DEVICE)

# Open the video file
cap = cv2.VideoCapture("/opt/videos/shopping_mall_01.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [ ]:
def draw_bbox(image, filtered_data):
    lx, ly, rx, ry = filtered_data[1:5]
    color = (0, 255, 0)
    cv2.rectangle(image, (int(lx), int(ly)), (int(rx), int(ry)), color, 2)
    cv2.putText(image, f"ID {filtered_data[0]}", (int(lx), int(ly) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    for i in range(5, len(filtered_data), 3):
        x = int(filtered_data[i])
        y = int(filtered_data[i + 1])
        cv2.circle(image, (x, y), 3, color, -1)

    return image

In [ ]:
# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Use a list to store the detected boxes
    dets_data = []

    # Process the pose
    poses = results_2_pose_list(results)

    # Apply NMS to the poses
    poses = nms_for_yolo_pose(poses)



    # Process the pose keypoints
    pose_data = results_2_pose_list(keypoints)

    # Perform object tracking
    tracked_objects = sort.update(boxes)

    # Draw the pose keypoints
    for pose in pose_data:
        frame = pose.draw(frame)

    # Display the frame
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break